In [3]:
# Import statements needed
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch import nn
from torch import optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import os
import torchvision
import time
import pandas as pd
import random
import copy
from tqdm import tqdm 
import torchvision.models as models
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [4]:
# Use GPU if applicable
print(torch.cuda.is_available())
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
device

True


'cuda'

In [5]:
train_path = "imagenette2/train"
test_path = "imagenette2/val"

In [6]:
transform = transforms.Compose([
    transforms.Resize((512, 512)),  
    transforms.ToTensor(),           
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
])


In [7]:
train = datasets.ImageFolder(root=train_path, transform=transform)
test = datasets.ImageFolder(root=test_path, transform=transform)

In [9]:
len(test)

3925

In [10]:
for x in train:
    print(x)
    break

(tensor([[[ 0.1608,  0.3490,  0.0275,  ...,  0.5373,  0.4667,  0.3961],
         [-0.2235, -0.1373,  0.0510,  ...,  0.5765,  0.5216,  0.4510],
         [-0.3961, -0.4588,  0.0510,  ...,  0.5843,  0.5451,  0.5137],
         ...,
         [-0.1765, -0.2078, -0.2157,  ..., -0.1686, -0.2000, -0.1922],
         [-0.2235, -0.2549, -0.2627,  ..., -0.2000, -0.2392, -0.2235],
         [-0.2784, -0.2941, -0.3098,  ..., -0.2392, -0.2549, -0.2549]],

        [[ 0.2706,  0.4118,  0.0902,  ...,  0.6000,  0.5451,  0.4745],
         [-0.1529, -0.0588,  0.1529,  ...,  0.6549,  0.6157,  0.5529],
         [-0.3647, -0.4039,  0.1137,  ...,  0.6706,  0.6471,  0.6078],
         ...,
         [ 0.1686,  0.1373,  0.1137,  ..., -0.1529, -0.1608, -0.1451],
         [ 0.0667,  0.0510,  0.0353,  ..., -0.1686, -0.1843, -0.1765],
         [-0.0118, -0.0196, -0.0275,  ..., -0.1843, -0.1922, -0.1922]],

        [[-0.3176, -0.1765, -0.2314,  ...,  0.5451,  0.4824,  0.4118],
         [-0.5451, -0.4824, -0.3882,  ...,  

In [6]:
def train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model):
    # variables needed for metrics later
    train_losses = []
    test_losses = 0
    train_accuracy = []
    test_accuracy = 0
    start_time_train = time.time()
    ############################ Train Loop ############################
    for epoch in range(epochs):
        # variables needed for metrics later
        train_size = len(train_dataloader.dataset)
        # makes sure to set model to train
        model.train()
        train_loss = 0
        train_correct = 0
        train_num_batches = len(train_dataloader)
        # Just to help with keep track of how long it taking
        train_loadbar = tqdm(train_dataloader, total=train_num_batches)
        for batch, (X, labels) in enumerate(train_loadbar):
            # Make sure values are on correct device
            X = X.to(device)
            labels = labels.to(device)

            # Model pred + loss
            pred = model(X)
            loss = loss_func(pred, labels)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Compute metrics
            train_loss+=loss.item()
            train_correct+=(pred.argmax(axis = 1) == labels).type(torch.float).sum().item()

            # Update the loading bar    
            train_loadbar.set_description(f'Epoch [{epoch + 1}/{epochs}]')
            train_loadbar.set_postfix(train_loss=train_loss/(batch + 1), train_accuracy=train_correct/train_size)


        # Compute metrics
        train_losses.append(train_loss/train_num_batches)
        train_accuracy.append(train_correct/train_size)

        end_time_train = time.time()
        train_time = end_time_train - start_time_train
        ############################ Train Loop ############################
        
        ############################ Test Loop #############################
        test_size = len(test_dataloader.dataset)
        test_num_batches = len(test_dataloader)
        # makes sure to set model to eval
        model.eval()
        # variables needed for metrics later
        start_time_test = time.time()
        test_loss = 0
        test_correct = 0
        test_loadbar = tqdm(test_dataloader, total=test_num_batches)
        with torch.no_grad():
            for batch, (X, labels) in enumerate(test_loadbar):
                # Make sure values are on correct device
                X = X.to(device)
                labels = labels.to(device)

                # Model pred + loss
                pred = model(X)
                loss = loss_func(pred, labels)

                # Compute metrics
                test_loss+=loss.item()
                test_correct+=(pred.argmax(axis = 1) == labels).type(torch.float).sum().item()
                test_loadbar.set_description(f'Epoch [{epoch + 1}/{epochs}]')
                test_loadbar.set_postfix(test_loss=test_loss/(batch + 1))
            # Compute metrics
            test_losses = test_loss/test_num_batches
            test_accuracy = test_correct/test_size
        print(f"Test Acc {test_accuracy}")
        
        end_time_test = time.time()
        test_time = end_time_test - start_time_test
        ############################ Test Loop #############################

    return train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time

In [7]:
# Compact high-level loop that runs everything and allows modification of all variables
#model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1', pretrained = True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# Make 10 class classifier
#model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
model.fc = nn.Linear(model.fc.in_features, 10)
model.to(device)
############################################# HYPER PARAMS #############################################
batch_size = 64
loss_func = nn.CrossEntropyLoss()
lr = .0001
weight_decay = .0001
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
epochs = 20
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=epochs/3, gamma=.5)
############################################# HYPER PARAMS #############################################
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)
train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time = train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model)
# Print results to table
data = [train_time, test_time, train_accuracy[-1], test_accuracy, 64, .0001]
#torch.save(model.state_dict(), f'models/Default_Model.pth')
print(data)

Using cache found in C:\Users\Shaan/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Shaan\miniconda3\envs\proj\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shaan\miniconda3\envs\proj\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/20]: 100%|██████████| 62/62 [00:40<00:00,  1.52it/s, test_loss=2.3] 


Test Acc 0.11847133757961784


Epoch [2/20]: 100%|██████████| 62/62 [00:30<00:00,  2.00it/s, test_loss=2.3] 


Test Acc 0.056560509554140125


Epoch [3/20]:  15%|█▍        | 22/148 [00:17<01:37,  1.29it/s, train_accuracy=0, train_loss=2.32]


KeyboardInterrupt: 

In [ ]:
print(data)

[541.7273671627045, 117.98300266265869, 0.8051366283410483, 0.29304166044497537, 64, 0.0001]
